[![GitHub](https://img.shields.io/badge/Github-hibana2077-blue?style=plastic-square&logo=github)](https://github.com/hibana2077)
[![Colab](https://img.shields.io/badge/Colab-Open%20in%20Colab-blue?style=plastic-square&logo=googlecolab)](https://colab.research.google.com/github/hibana2077/hibana2077/blob/master/train/train.ipynb)

In [ ]:
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
# !tar -xzvf ta-lib-0.4.0-src.tar.gz
# %cd ta-lib
# !./configure --prefix=/usr
# !make
# !make install
# !pip install Ta-Lib

In [1]:
#using sklearn random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from talib import abstract
from ccxt import binance,bybit
import pandas as pd
import numpy as np
import os

In [2]:
os.remove("../data/BTC_USDT_15m.csv")

In [3]:
binance = binance()
symbol = 'BTC/USDT'
timeframe = '15m'
file_name = f"../data/{symbol.replace('/', '_')}_{timeframe}.csv"
start = binance.parse8601('2019-10-01T00:00:00Z')
end = binance.parse8601('2022-12-30T00:00:00Z')
cnt_time = start
data = []
while cnt_time < end:
    ohlcv = binance.fetch_ohlcv(symbol, timeframe, cnt_time)
    data += ohlcv
    cnt_time = ohlcv[-1][0] + 900000 # 1h in ms    
df = pd.DataFrame(data, columns=['time', 'open', 'high', 'low', 'close', 'volume'])
df['time'] = pd.to_datetime(df['time'], unit='ms')
df.to_csv(file_name, index=False)

In [4]:
df

,time,open,high,low,close,volume
0,2019-10-01 00:00:00,8289.97,8310.08,8206.57,8243.39,1016.103644
1,2019-10-01 00:15:00,8242.94,8252.53,8210.01,8220.39,391.576024
2,2019-10-01 00:30:00,8220.78,8222.13,8188.00,8213.23,285.706881
3,2019-10-01 00:45:00,8213.23,8218.34,8195.36,8199.66,176.788254
4,2019-10-01 01:00:00,8199.21,8222.28,8173.05,8211.85,420.860704
...,...,...,...,...,...,...
113905,2023-01-01 05:30:00,16521.42,16522.75,16514.79,16520.02,669.280250
113906,2023-01-01 05:45:00,16520.03,16534.98,16518.85,16530.71,999.506850
113907,2023-01-01 06:00:00,16530.71,16543.69,16528.81,16537.72,949.233190
113908,2023-01-01 06:15:00,16538.05,16550.00,16537.64,16544.63,976.155330


In [5]:
df['MFI'] = abstract.MFI(df, timeperiod=14)
df['RSI'] = abstract.RSI(df, timeperiod=14)
df['ADX'] = abstract.ADX(df, timeperiod=14)
df['CCI'] = abstract.CCI(df, timeperiod=14)
df['ATR'] = abstract.ATR(df, timeperiod=14)
df['OBV'] = abstract.OBV(df, timeperiod=14)
df['EMA'] = abstract.EMA(df, timeperiod=14)
df['WILLR'] = abstract.WILLR(df, timeperiod=14)
df['AD'] = abstract.AD(df, timeperiod=14)
df['ADOSC'] = abstract.ADOSC(df, timeperiod=14)
df['ADXR'] = abstract.ADXR(df, timeperiod=14)
df['APO'] = abstract.APO(df, timeperiod=14)
df['AROONOSC'] = abstract.AROONOSC(df, timeperiod=14)
df['BOP'] = abstract.BOP(df, timeperiod=14)
df['CCI'] = abstract.CCI(df, timeperiod=14)
df['CMO'] = abstract.CMO(df, timeperiod=14)
df['DX'] = abstract.DX(df, timeperiod=14)
df['MOM'] = abstract.MOM(df, timeperiod=14)
df['PPO'] = abstract.PPO(df, timeperiod=14)
df['ROC'] = abstract.ROC(df, timeperiod=14)
df['ROCP'] = abstract.ROCP(df, timeperiod=14)
df['ROCR'] = abstract.ROCR(df, timeperiod=14)
df['ROCR100'] = abstract.ROCR100(df, timeperiod=14)
df['RSI'] = abstract.RSI(df, timeperiod=14)
df['TRIX'] = abstract.TRIX(df, timeperiod=14)
df['ULTOSC'] = abstract.ULTOSC(df, timeperiod=14)
df['WILLR'] = abstract.WILLR(df, timeperiod=14)
df['WMA'] = abstract.WMA(df, timeperiod=14)
df['HT_TRENDLINE'] = abstract.HT_TRENDLINE(df, timeperiod=14)
df['TRANGE'] = abstract.TRANGE(df, timeperiod=14)
df['CLOSE_percent'] = df['close'].pct_change()

In [6]:
df['UP'] = df['CLOSE_percent'].apply(lambda x: 1 if x > 0 else 0)
df['DOWN'] = df['CLOSE_percent'].apply(lambda x: 1 if x < 0 else 0)
df['UP'] = df['UP'].shift(-1) #shift UP DOWN 一個單位，因為我們要預測的是下一個時間點的漲跌
df['DOWN'] = df['DOWN'].shift(-1)

df = df.dropna()

In [7]:
#正規化
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['MFI','RSI','ADX','CCI','ATR','OBV','EMA','WILLR','AD','ADOSC','ADXR','APO','AROONOSC','BOP','CCI','CMO','DX','MOM','PPO','ROC','ROCP','ROCR','ROCR100','RSI','TRIX','ULTOSC','WILLR','WMA','HT_TRENDLINE','TRANGE']] = scaler.fit_transform(df[['MFI','RSI','ADX','CCI','ATR','OBV','EMA','WILLR','AD','ADOSC','ADXR','APO','AROONOSC','BOP','CCI','CMO','DX','MOM','PPO','ROC','ROCP','ROCR','ROCR100','RSI','TRIX','ULTOSC','WILLR','WMA','HT_TRENDLINE','TRANGE']])
df[['open','high','low','close','volume']] = scaler.fit_transform(df[['open','high','low','close','volume']])

C:\Users\李軒豪\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\李軒豪\AppData\Roaming\Python\Python310\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [8]:
X = df.drop(['UP','DOWN','time','CLOSE_percent'], axis=1)
#y -> UP and DOWN
y = df['UP'] #UP

In [9]:
X

,open,high,low,close,volume,MFI,RSI,ADX,CCI,ATR,...,PPO,ROC,ROCP,ROCR,ROCR100,TRIX,ULTOSC,WMA,HT_TRENDLINE,TRANGE
63,0.068547,0.063779,0.070296,0.069147,0.009006,0.545123,0.573676,0.110529,0.663657,0.017247,...,0.665069,0.525270,0.525270,0.525270,0.525270,0.652433,0.483833,0.060348,0.056874,0.005730
64,0.069061,0.063711,0.070232,0.068538,0.005545,0.583632,0.478211,0.097099,0.543233,0.017270,...,0.663999,0.521721,0.521721,0.521721,0.521721,0.652979,0.423186,0.060335,0.056875,0.005697
65,0.068445,0.063783,0.070170,0.069236,0.006517,0.645869,0.569353,0.087159,0.630719,0.017612,...,0.664184,0.530577,0.530577,0.530577,0.530577,0.654181,0.571219,0.060415,0.056885,0.006910
66,0.069142,0.065076,0.070752,0.070208,0.028755,0.666238,0.661885,0.109944,0.828155,0.019634,...,0.666379,0.536803,0.536803,0.536803,0.536803,0.656909,0.610421,0.060617,0.056919,0.013368
67,0.070112,0.065230,0.071492,0.070131,0.014693,0.677433,0.650392,0.133758,0.750029,0.020111,...,0.668514,0.536073,0.536073,0.536073,0.536073,0.660414,0.595304,0.060796,0.056964,0.008062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113904,0.194818,0.189655,0.196764,0.194934,0.025318,0.234360,0.421736,0.425441,0.401437,0.004835,...,0.662153,0.516953,0.516953,0.516953,0.516953,0.655422,0.429195,0.188287,0.185677,0.001451
113905,0.194859,0.189656,0.196802,0.194917,0.016578,0.287359,0.416680,0.434927,0.416069,0.004536,...,0.661421,0.517084,0.517084,0.517084,0.517084,0.655338,0.442788,0.188263,0.185667,0.001116
113906,0.194838,0.189845,0.196865,0.195082,0.024758,0.297335,0.483630,0.419165,0.500923,0.004561,...,0.660866,0.517732,0.517732,0.517732,0.517732,0.655379,0.524410,0.188266,0.185665,0.002261
113907,0.195003,0.189980,0.197019,0.195190,0.023513,0.364064,0.522961,0.390650,0.601020,0.004538,...,0.661242,0.519734,0.519734,0.519734,0.519734,0.655573,0.614233,0.188287,0.185664,0.002086


In [10]:
y.value_counts()

1.0    57270
0.0    56576
Name: UP, dtype: int64

In [11]:
X = X.values
y = y.values

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [13]:
X_train

array([[0.45660004, 0.45706938, 0.45486613, ..., 0.45824923, 0.46015099,
        0.08820994],
       [0.46206755, 0.45785413, 0.46366982, ..., 0.45713222, 0.45609097,
        0.01548531],
       [0.06884395, 0.06353146, 0.07034482, ..., 0.06126379, 0.05825312,
        0.00304351],
       ...,
       [0.04843619, 0.04305331, 0.04948957, ..., 0.04031012, 0.03686429,
        0.00628749],
       [0.65859559, 0.65750067, 0.66158932, ..., 0.65500844, 0.65230799,
        0.03286739],
       [0.27439417, 0.27277663, 0.27627633, ..., 0.27078841, 0.26937049,
        0.03489873]])

In [14]:
#random forest classifier with n_estimators=10 (default)
model = RandomForestClassifier(
    n_estimators=128)



In [15]:
#train the model using the training sets y_pred=clf.predict(X_test)
model.fit(X_train,y_train)

RandomForestClassifier(n_estimators=128)

In [16]:
pred = model.predict(X_test)

print(accuracy_score(y_test, pred))

0.5271941536083199


In [17]:
pred

array([1., 1., 1., ..., 0., 1., 1.])

In [18]:
X_train[0]

array([0.45660004, 0.45706938, 0.45486613, 0.45947973, 0.06942217,
       0.51681164, 0.43555135, 0.14727653, 0.39895208, 0.16609507,
       0.46466092, 0.45824186, 0.27686228, 0.79925654, 0.49706931,
       0.15619416, 0.54233311, 0.07142857, 0.64559296, 0.43555135,
       0.05408606, 0.49838958, 0.64838202, 0.50328912, 0.50328912,
       0.50328912, 0.50328912, 0.64884387, 0.43747946, 0.45824923,
       0.46015099, 0.08820994])

In [19]:
test_ans = model.predict(X_train)
print(accuracy_score(y_train, test_ans))

1.0


In [20]:
#confusion matrix
confusion_matrix(y_test, pred)


array([[7555, 6650],
       [6807, 7450]], dtype=int64)

In [21]:
#classification report  
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

         0.0       0.53      0.53      0.53     14205
         1.0       0.53      0.52      0.53     14257

    accuracy                           0.53     28462
   macro avg       0.53      0.53      0.53     28462
weighted avg       0.53      0.53      0.53     28462



In [22]:
#ROC curve
fpr, tpr, thresholds = roc_curve(y_test, pred)
auc = roc_auc_score(y_test, pred)
print('AUC: %.2f' % auc)

AUC: 0.53


In [23]:
#save model
import pickle
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))
